
# Competition Information:
This is the second Microsoft Malware Hosted competition on Kaggle. The former can be found [here](https://www.kaggle.com/c/malware-classification). Let us see what's this competition is all about..

![](https://zdnet1.cbsistatic.com/hub/i/2014/08/27/0d77a99a-2da9-11e4-9e6a-00505685119a/1a84d511ed7039462265cda82825e113/microsofts-logo-gets-a-makeover.jpg)
The malware industry continues to be a well-organized, well-funded market dedicated to evading traditional security measures. Once a computer is infected by malware, criminals can hurt consumers and enterprises in many ways.With more than one billion enterprise and consumer customers, Microsoft takes this problem very seriously and is deeply invested in improving security.
As one part of their overall strategy for doing so, Microsoft is challenging the data science community to develop techniques to predict if a machine will soon be hit with malware. As with their previous, Malware Challenge (2015), Microsoft is providing Kagglers with an unprecedented malware dataset to encourage open-source progress on effective techniques for predicting malware occurrences.



## Goals 
The goal of this competition is to predict a Windows machine’s probability of getting infected by various families of malware, based on different properties of that machine. The telemetry data containing these properties and the machine infections was generated by combining heartbeat and threat reports collected by Microsoft's endpoint protection solution, Windows Defender. Each row in this dataset corresponds to a machine, uniquely identified by a MachineIdentifier. HasDetections is the ground truth and indicates that Malware was detected on the machine. Using the information and labels in train.csv, you must predict the value for HasDetections for each machine in test.csv.

## Metric

Submissions are evaluated on [area under the ROC curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) between the predicted probability and the observed label.


Let's check what tools and files we have been given to recipe our model..

In [ ]:
# Imports And Libraries
import time
import warnings
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import gc
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from functools import partial, wraps
from datetime import datetime as dt
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
import itertools

def get_random_pairs(numbers):
    pairs = list(itertools.combinations(numbers, 2))
    return list(pairs)
def autobox(cols):
    length = len(list(cols)) + 1
    if length == 1:
        m, n = 2, 1
    else:
        m = int(length**0.5)
        n = int(length / m)
    return n, m + 1
def crosstab_all(df,
                 cat_cols,
                 annot=True,
                 cmap='viridis',
                 figsize=(20, 15),
                 subplot_grid=None,
                 title='Paired Chi Square Matrices',
                 **kwargs):
    plt.figure(figsize=figsize)
    plt.suptitle(title, x=0.5, y=0.9, size=20)
    i = 0
    pairs = get_random_pairs(cat_cols)
    m, n = autobox(cat_cols)
    factor = len(pairs)
    m = int(factor / 3) + 2
    if subplot_grid:
        m = subplot_grid[0]
        n = subplot_grid[1]
    for pair in pairs:
        ax = plt.subplot(m, n, i + 1)
        ax.set_xticklabels(
            ax.get_xticklabels(), rotation=60, ha="right", fontsize=13)
        x = df[pair[0]]
        y = df[pair[1]]
        sns.heatmap(
            pd.crosstab(x, y).apply(lambda r: r / r.sum(), axis=1),
            annot=annot,
            cmap=cmap,
            **kwargs)
        i += 1

In [ ]:
!ls ../input

In [ ]:
# Credits @Theol
dtypes = {'MachineIdentifier': 'category', 'ProductName': 'category', 'EngineVersion': 'category', 'AppVersion': 'category', 'AvSigVersion': 'category', 'IsBeta': 'int8', 'RtpStateBitfield': 'float16', 'IsSxsPassiveMode': 'int8', 'DefaultBrowsersIdentifier': 'float16', 'AVProductStatesIdentifier': 'float32', 'AVProductsInstalled': 'float16', 'AVProductsEnabled': 'float16', 'HasTpm': 'int8', 'CountryIdentifier': 'int16', 'CityIdentifier': 'float32', 'OrganizationIdentifier': 'float16', 'GeoNameIdentifier': 'float16', 'LocaleEnglishNameIdentifier': 'int8', 'Platform': 'category', 'Processor': 'category', 'OsVer': 'category', 'OsBuild': 'int16', 'OsSuite': 'int16', 'OsPlatformSubRelease': 'category', 'OsBuildLab': 'category', 'SkuEdition': 'category', 'IsProtected': 'float16', 'AutoSampleOptIn': 'int8', 'PuaMode': 'category', 'SMode': 'float16', 'IeVerIdentifier': 'float16', 'SmartScreen': 'category', 'Firewall': 'float16', 'UacLuaenable': 'float32', 'Census_MDC2FormFactor': 'category', 'Census_DeviceFamily': 'category', 'Census_OEMNameIdentifier': 'float16', 'Census_OEMModelIdentifier': 'float32', 'Census_ProcessorCoreCount': 'float16', 'Census_ProcessorManufacturerIdentifier': 'float16', 'Census_ProcessorModelIdentifier': 'float16', 'Census_ProcessorClass': 'category', 'Census_PrimaryDiskTotalCapacity': 'float32', 'Census_PrimaryDiskTypeName': 'category', 'Census_SystemVolumeTotalCapacity': 'float32', 'Census_HasOpticalDiskDrive': 'int8', 'Census_TotalPhysicalRAM': 'float32', 'Census_ChassisTypeName': 'category', 'Census_InternalPrimaryDiagonalDisplaySizeInInches': 'float16', 'Census_InternalPrimaryDisplayResolutionHorizontal': 'float16', 'Census_InternalPrimaryDisplayResolutionVertical': 'float16', 'Census_PowerPlatformRoleName': 'category', 'Census_InternalBatteryType': 'category', 'Census_InternalBatteryNumberOfCharges': 'float32', 'Census_OSVersion': 'category', 'Census_OSArchitecture': 'category', 'Census_OSBranch': 'category', 'Census_OSBuildNumber': 'int16', 'Census_OSBuildRevision': 'int32', 'Census_OSEdition': 'category', 'Census_OSSkuName': 'category', 'Census_OSInstallTypeName': 'category', 'Census_OSInstallLanguageIdentifier': 'float16', 'Census_OSUILocaleIdentifier': 'int16', 'Census_OSWUAutoUpdateOptionsName': 'category', 'Census_IsPortableOperatingSystem': 'int8', 'Census_GenuineStateName': 'category', 'Census_ActivationChannel': 'category', 'Census_IsFlightingInternal': 'float16', 'Census_IsFlightsDisabled': 'float16', 'Census_FlightRing': 'category', 'Census_ThresholdOptIn': 'float16', 'Census_FirmwareManufacturerIdentifier': 'float16', 'Census_FirmwareVersionIdentifier': 'float32', 'Census_IsSecureBootEnabled': 'int8', 'Census_IsWIMBootEnabled': 'float16', 'Census_IsVirtualDevice': 'float16', 'Census_IsTouchEnabled': 'int8', 'Census_IsPenCapable': 'int8', 'Census_IsAlwaysOnAlwaysConnectedCapable': 'float16', 'Wdft_IsGamer': 'float16', 'Wdft_RegionIdentifier': 'float16', 'HasDetections': 'int8'}


To make the analysis simple. I am taking only first **50,000** rows from the dataset

In [ ]:
# Imports 
train_df = pd.read_csv("../input/train.csv", dtype= dtypes, nrows=50000)
test_df = pd.read_csv("../input/test.csv", dtype = dtypes, nrows=50000)
print("Train Shape : ", train_df.shape)
print("Test Shape : ", test_df.shape)

Let's print out and take a look at the header frame

In [ ]:
train_df.head()

In [ ]:
def mr_inspect(df):
    """Returns a inspection dataframe"""
    inspect_dataframe = pd.DataFrame({'DataType': df.dtypes, 'Unique values': df.nunique() ,
                  'Number of missing values': df.isnull().sum() ,
                  'Percentage missing': (df.isnull().sum() / len(df)) * 100,
                                      'Memory Usage (MB)':round(df.memory_usage(index=False) / 1024, 2)
                                     }).sort_values(by='Number of missing values', ascending = False)
    inspect_dataframe['Variance'] = df[inspect_dataframe.index].var()
    inspect_dataframe['Mean'] = df[inspect_dataframe.index].mean()
    inspect_dataframe['Min'] = df[inspect_dataframe.index].min()
    inspect_dataframe['Max'] = df[inspect_dataframe.index].max()
    return inspect_dataframe

Huge datasets and many columns is always a bummer to eyes. But, statistics and pretty tabular conversion makes much sense. Let's study out such inspection of data types and values statistics for the features.
given. The code above prints out information like;

1. Data Type
2. Unique Values
3. Missing Values
4. Percentage Of Missing
5. Memory Usage (MB)

And few statistical features ...

6. Variance
7. Mean
8. Min
9. Max

## Train Inspection Frame

In [ ]:
ins_train = mr_inspect(train_df)
ins_train.transpose()

## Test Inspection Frame

In [ ]:
mr_inspect(test_df).transpose()

## Target Analysis

In [ ]:
target = train_df['HasDetections']
cnt_srs = train_df['HasDetections'].value_counts()
trace = go.Bar(
    x=cnt_srs.index,
    y=cnt_srs.values,
    marker=dict(
        color=cnt_srs.values,
        colorscale = 'Picnic',
        reversescale = True
    ),
)

layout = go.Layout(
    title='Target Count',
    font=dict(size=18))

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="TargetCount")

In [ ]:
labels = (np.array(cnt_srs.index))
sizes = (np.array((cnt_srs / cnt_srs.sum())*100))

trace = go.Pie(labels=labels, values=sizes)
layout = go.Layout(
    title='Target distribution',
    font=dict(size=18),
    width=600,
    height=600)
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,)

Great, now we have the challenge to build models so that we can detect Malware occurances for a particular specifications. To get started let us identify columns in our dataset

## Identifying And Grouping Columns

In [ ]:
categorical_cols = list(ins_train[ins_train['DataType'] == 'category'].index)
numerical_cols = list(ins_train[ins_train['DataType'] != 'category'].index)
binary_cols = []
for col in (numerical_cols):
    if train_df[col].nunique() == 2:
        binary_cols.append(col)
        numerical_cols.remove(col)

for col in (categorical_cols):
    if train_df[col].nunique() == 2:
        binary_cols.append(col)
        categorical_cols.remove(col)
        
print(">> Categorical Columns: {}: \n {} \n\n".format(len(categorical_cols), categorical_cols))
print(">> Numerical Columns: {}: \n{}\n\n".format(len(numerical_cols), numerical_cols))
print(">> Binary Columns: {}:\n{}".format(len(binary_cols), binary_cols))

Rich information. Exactly what we need to make   robust machine learning models. Given the dimentionality of the dataset we will proceed analysis after taking importances by training a public baseline model

## Baseline Model And Feature Importances

In [ ]:
params = {'objective': 'binary', 'boosting_type': 'gbdt', 'learning_rate': 0.02, 'max_depth': 8, 'num_leaves': 67, 'n_estimators': 1000, 'bagging_fraction': 0.4, 'feature_fraction': 0.5, 'bagging_freq': 5, 'bagging_seed': 2018, 'min_child_samples': 80, 'min_child_weight': 100.0, 'min_split_gain': 0.1, 'reg_alpha': 0.005, 'reg_lambda': 0.1, 'subsample_for_bin': 25000, 'min_data_per_group': 100, 'max_cat_to_onehot': 4, 'cat_l2': 25.0, 'cat_smooth': 2.0, 'max_cat_threshold': 32, 'random_state': 1, 'silent': True, 'metric': 'auc'}
# Credit @jiazhuang
TARGET = 'HasDetections'
TARGET_INDEX = 'MachineIdentifier'
def modeling_cross_validation(params, X, y, folds=2):
    clfs = list()
    oof_preds = np.zeros(X.shape[0])
    # Split data with kfold
    kfolds = StratifiedKFold(n_splits=folds, shuffle=False, random_state=42)
    for n_fold, (trn_idx, val_idx) in enumerate(kfolds.split(X, y)):
        X_train, y_train = X.iloc[trn_idx], y.iloc[trn_idx]
        X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]

        # LightGBM Regressor estimator
        model = lgb.LGBMClassifier(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            verbose=-1, eval_metric='auc',
            early_stopping_rounds=100
        )

        clfs.append(model)
        oof_preds[val_idx] = model.predict(X_valid, num_iteration=model.best_iteration_)
        
    score = roc_auc_score(y, oof_preds)
    print(score)
    return clfs, score

def get_importances(clfs, feature_names):
    importances = pd.DataFrame()
    for i, model in enumerate(clfs, 1):
        # Feature importance
        imp_df = pd.DataFrame({
                "feature": feature_names, 
                "gain": model.booster_.feature_importance(importance_type='gain'),
                "fold": model.n_features_,
                })
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

    importances['gain_log'] = importances['gain']
    mean_gain = importances[['gain', 'feature']].groupby('feature').mean()
    importances['mean_gain'] = importances['feature'].map(mean_gain['gain'])
    importances.sort_values(by='gain', inplace=True, ascending=False)
    importances.to_csv('feature_importances.csv', index=False)
    return importances

def predict_cross_validation(test, clfs):
    sub_preds = np.zeros(test.shape[0])
    for i, model in enumerate(clfs, 1):    
        test_preds = model.predict_proba(test, num_iteration=model.best_iteration_)
        sub_preds += test_preds[:,1]

    sub_preds = sub_preds / len(clfs)
    ret = pd.Series(sub_preds, index=test.index)
    ret.index.name = test.index.name
    return ret
def predict_test_chunk(features, clfs, dtypes, filename='tmp.csv', chunks=100000):
    
    for i_c, df in enumerate(pd.read_csv('../input/test.csv', 
                                         chunksize=chunks, 
                                         dtype=dtypes, 
                                         iterator=True)):
        
        df.set_index(TARGET_INDEX, inplace=True)
        preds_df = predict_cross_validation(df[features], clfs)
        preds_df = preds_df.to_frame(TARGET)

        if i_c == 0:
            preds_df.to_csv(filename, header=True, mode='a', index=True)
        else:
            preds_df.to_csv(filename, header=False, mode='a', index=True)
        del preds_df
        gc.collect()
train_features = list()
train = pd.read_csv('../input/train.csv', nrows=50000, dtype=dtypes).set_index(TARGET_INDEX)
train_features = [f for f in train.columns if f != TARGET]
clfs, score = modeling_cross_validation(params, train[train_features], train[TARGET], folds=5)

In [ ]:
feature_importances = get_importances(clfs, train_features)
feature_importances.drop_duplicates(subset=['feature'], inplace=True)

In [ ]:
data = [go.Bar(
            x= feature_importances.feature.values,
            y= feature_importances.gain.values, marker = dict(
          color = 'gold'
        ), orientation = 'v'
    )]
layout = go.Layout(
    title='Feature Importances LGB Model')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Now, that we have clear picture of what's important, let us extract these and find their story. Priting Top 40 features that model discovered and head of train.csv with only important ones

In [ ]:
top40 = list(feature_importances['feature'][:40])
print(">> Top 40 Features are:\n", top40)
feature_importances.head(40).set_index('feature').transpose()

In [ ]:
train_df[top40].head(20)

It occurs to me that the following features were well selective for a particular version release. 

**These features actually help us identify the software version/ commits that might have introduced the vulnerability or loop holes in the OS**

## Performing Chi2 Tests 

We can plot the chi square matrix with simple crosstab function from pandas. Let us use this to make a custom crosstab_all function which takes a list of categorical colum and then generate chi2 matrices for all possible pairs. (Code Hidden In #2 Cell of Notebook)

In [ ]:
crosstab_list = ['OsBuild', 'HasDetections', 'Census_OSVersion','ProductName']
print("CrossTab for:", crosstab_list)
crosstab_all(train_df,crosstab_list, cmap='summer', figsize=(20,18), subplot_grid=(4,3), annot=False)

In [ ]:
crosstab_list = [ 'Census_OSVersion', 'SmartScreen','HasDetections']
print("CrossTab for:", crosstab_list)
crosstab_all(train_df,crosstab_list, cmap='coolwarm', figsize=(27,15), subplot_grid=(2,3), annot=False)

In [ ]:
crosstab_list = ['OsVer', 'Platform', 'Processor', 'HasDetections']
print("CrossTab for:", crosstab_list)
crosstab_all(train_df,crosstab_list, cmap='winter', figsize=(20,18))

In [ ]:
crosstab_list = ['Census_ProcessorClass', 'HasDetections', 'SmartScreen','ProductName']
print("CrossTab for:", crosstab_list)
crosstab_all(train_df,crosstab_list, cmap='plasma', figsize=(20,18), subplot_grid=(4,3))

## HasDetection Vs. NoDetections

In [ ]:
HasDetections = train_df[train_df.HasDetections == 1]
NoDetections = train_df[train_df.HasDetections == 0]

In [ ]:
def countPlot(key, title):
    trace1 = go.Bar(
        x= list(dict(HasDetections[key].value_counts()).keys()), 
        y= list(dict(HasDetections[key].value_counts()).values()),
        name='HasDetections'
    )
    trace2 = go.Bar(
        x= list(dict(NoDetections[key].value_counts()).keys()),
        y= list(dict(NoDetections[key].value_counts()).values()),
        name='NoDetections'
    )

    data = [trace1, trace2]
    layout = go.Layout(
        barmode='group', title=title
    )
    fig = go.Figure(data=data, layout=layout)
    return py.iplot(fig)

In [ ]:
train_df[top40].head(20)

In [ ]:
countPlot('SmartScreen', "CountPlot: SmartScreen")

In [ ]:
countPlot('Census_ChassisTypeName', "CountPlot: Census_ChassisTypeName")

In [ ]:
countPlot('AVProductsInstalled', "CountPlot: AVProductsInstalled")

## Multicorrelation Matrix

In [ ]:
def multi_corr_heatmap(df,annot=True,
                       cmap='viridis',
                       figsize=(15, 10),
                       title='Numerical Columns Correlation Heatmap',
                       **kwargs):
    plt.figure(figsize=figsize)
    plt.suptitle(title, x=0.5, y=.94, size=20)
    sns.heatmap(df.corr(), annot=annot, cmap=cmap, **kwargs)
multi_corr_heatmap(train_df[numerical_cols], cmap='Blues_r', annot=False, vmax=0.9)

## Training With Top 50 Features For Submission

In [ ]:
train_features = list(feature_importances.feature[:50])
clfs, score = modeling_cross_validation(params, train[train_features], train[TARGET], folds=5)
filename = 'shaz13_submission_baseline.csv'
predict_test_chunk(train_features, clfs, dtypes, filename=filename, chunks=100000)

**More to come. Stay Tuned** 

I highly appreciate your feedbacks on this. Do let me know if you like more EDA or Baseline tuning from this point. Thanks! 

Peace. shaz13